In [1]:
import pandas as pd
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import pdb
import scipy
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from scipy.optimize import basinhopping
from tqdm.notebook import tqdm
import math

In [2]:
""" 
Obtaining data from a given expt
"""
csv_test = pd.read_csv('../auditory_categorization_longLow/important_things_not_included_in_assets/allTrials.csv');
csv_data = pd.read_csv('auditory_categorization_lc_online_data/auditory_categorization_v2_119865_2021-06-11_19h04.17_d1d63a32-cae7-11eb-abb1-ac1f6b405aea/609dd2bc3360f6e164901570_categorization_task_longLow_2021-06-11_13h42.28.848.csv');
        

In [3]:
n_tones = 3
n_trials = csv_data.shape[0]-47

"""
Get tones and values of keys pressed
"""
test_columns = list(csv_test.columns)
test_tones_name = test_columns.index('Name')
test_tones_col_idx = test_columns.index('Tones')
df_names = (csv_test.iloc[0:800,test_tones_name]).values
df_tones = (csv_test.iloc[0:800,test_tones_col_idx]).values

tones_array_orig = np.zeros((n_trials,n_tones))
tones_array_idxs_keep = []

for i_wav in range(804):
    if isinstance(csv_data['Name'][i_wav+46],str):
        tones_array_orig[i_wav,:] = np.array(df_tones[np.where(csv_data['Name'][i_wav+46]\
                                                          ==df_names)[0]][0][1:-1].split(',')).astype(float)  
        tones_array_idxs_keep += [i_wav]
        
df_tones = np.copy(tones_array_orig[tones_array_idxs_keep,:])
df_corrans = np.copy(csv_data['corrAns'][46:csv_data.shape[0]])[tones_array_idxs_keep]
df_keys = np.copy(csv_data['test_resp.keys'][46:csv_data.shape[0]])[tones_array_idxs_keep]

In [4]:
"""
Find no response cases in the expt
"""
no_response = np.intersect1d(np.where(df_keys!='h')[0],np.where(df_keys!='l')[0])
print("Did not respond to: ",no_response)

"""
Convert keys ['l','h'] to [0,1] and plot p(H|T)
"""
corrans_num_orig = np.zeros_like(df_corrans)
corrans_num_orig[df_corrans == 'h'] = 1

keys_num_orig = np.zeros_like(df_keys)
keys_num_orig[df_keys == 'h'] = 1

corrans_num_160 = corrans_num_orig[:160]
keys_num_160 = keys_num_orig[:160]
tones_array_160 = df_tones[:160]
print("Got correct: ", np.sum(keys_num_160==corrans_num_160)/len(tones_array_160))
print("Got high correct: ", np.sum((keys_num_160)*(corrans_num_160))/np.sum(corrans_num_160))
print("Got low correct: ", np.sum((1-keys_num_160)*(1-corrans_num_160))/np.sum(1-corrans_num_160))

corrans_num_320 = corrans_num_orig[160:320]
keys_num_320 = keys_num_orig[160:320]
tones_array_320 = df_tones[160:320]
print("Got correct: ", np.sum(keys_num_320==corrans_num_320)/len(tones_array_320))
print("Got high correct: ", np.sum((keys_num_320)*(corrans_num_320))/np.sum(corrans_num_320))
print("Got low correct: ", np.sum((1-keys_num_320)*(1-corrans_num_320))/np.sum(1-corrans_num_320))

corrans_num_480 = corrans_num_orig[320:480]
keys_num_480 = keys_num_orig[320:480]
tones_array_480 = df_tones[320:480]
print("Got correct: ", np.sum(keys_num_480==corrans_num_480)/len(tones_array_480))
print("Got high correct: ", np.sum((keys_num_480)*(corrans_num_480))/np.sum(corrans_num_480))
print("Got low correct: ", np.sum((1-keys_num_480)*(1-corrans_num_480))/np.sum(1-corrans_num_480))

corrans_num_640 = corrans_num_orig[480:640]
keys_num_640 = keys_num_orig[480:640]
tones_array_640 = df_tones[480:640]
print("Got correct: ", np.sum(keys_num_640==corrans_num_640)/len(tones_array_640))
print("Got high correct: ", np.sum((keys_num_640)*(corrans_num_640))/np.sum(corrans_num_640))
print("Got low correct: ", np.sum((1-keys_num_640)*(1-corrans_num_640))/np.sum(1-corrans_num_640))

corrans_num_800 = corrans_num_orig[640:800]
keys_num_800 = keys_num_orig[640:800]
tones_array_800 = df_tones[640:800]
print("Got correct: ", np.sum(keys_num_800==corrans_num_800)/len(tones_array_800))
print("Got high correct: ", np.sum((keys_num_800)*(corrans_num_800))/np.sum(corrans_num_800))
print("Got low correct: ", np.sum((1-keys_num_800)*(1-corrans_num_800))/np.sum(1-corrans_num_800))

corrans_num = corrans_num_orig[:800]
keys_num = keys_num_orig[:800]
tones_array = df_tones[:800]
print("Got correct: ", np.sum(keys_num==corrans_num)/len(tones_array))
print("Got high correct: ", np.sum((keys_num)*(corrans_num))/np.sum(corrans_num))
print("Got low correct: ", np.sum((1-keys_num)*(1-corrans_num))/np.sum(1-corrans_num))

Did not respond to:  []
Got correct:  0.89375
Got high correct:  0.8636363636363636
Got low correct:  0.9051724137931034
Got correct:  0.8
Got high correct:  0.7962962962962963
Got low correct:  0.8018867924528302
Got correct:  0.83125
Got high correct:  0.84
Got low correct:  0.8272727272727273
Got correct:  0.875
Got high correct:  0.8372093023255814
Got low correct:  0.8888888888888888
Got correct:  0.9
Got high correct:  0.9111111111111111
Got low correct:  0.8956521739130435
Got correct:  0.86
Got high correct:  0.847457627118644
Got low correct:  0.8652482269503546


In [5]:
trial_tones = np.repeat(tones_array,1,axis = 0)
trial_behaviour = np.reshape(keys_num,np.prod(keys_num.shape)) 
corrans_expt = np.reshape(corrans_num,np.prod(corrans_num.shape)) 
# this has been changed to check how values change with observer responses

expt_tones = np.arange(90,3000,1) #array of possible true tones
log_freq_seq_array = np.arange(0.6,4.7,0.1)
log_freq_percept = np.arange(0.6,4.7,0.1) # array of possible perceptual tones

idxs_with_response = np.delete(np.arange(len(trial_tones)),no_response)
trial_tones = trial_tones[idxs_with_response,:]
trial_behaviour = trial_behaviour[idxs_with_response]
corrans_expt = corrans_expt[idxs_with_response]

log_freq_percept, log_freq_seq_array.shape

(array([0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8,
        1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1,
        3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4,
        4.5, 4.6]),
 (41,))

In [6]:
"""
Using data to constrain the plow parameter
"""
idxHigh = np.arange(len(trial_behaviour))[corrans_expt==1]
idxLow = np.arange(len(trial_behaviour))[corrans_expt==0]
idxToKeep = np.concatenate((idxHigh, np.random.choice(idxLow,size=len(idxLow),replace=False)))
print("probability of low given the data is likely to be: ", 1-sum(trial_behaviour[idxToKeep])/len(idxToKeep))


probability of low given the data is likely to be:  0.655


In [7]:
def gaussian(x, mean, sigma):
    return np.exp(-(x-mean)**2/(2*sigma**2))

def Tones3dgrid(latentTones, sigma):    
    
    input_array_0 = np.expand_dims(gaussian(log_freq_percept, latentTones[0], sigma), axis = 1)
    input_array_1 = np.expand_dims(gaussian(log_freq_percept, latentTones[1], sigma), axis = 1)
    input_array_2 = np.expand_dims(gaussian(log_freq_percept, latentTones[2], sigma), axis = 1)
    s0 = 1/np.sum(input_array_0); s1 = 1/np.sum(input_array_1); s2 = 1/np.sum(input_array_2)
    input_array_0 *= s0; input_array_1 *= s1; input_array_2 *= s2; 
    
    input_array_mat = np.expand_dims(input_array_0@input_array_1.T,axis=2)@(input_array_2.T) #p(T1,T2..|H)     
    
    return input_array_mat

def posterior_array(freq_input, n_tones, p_back, p_low, log_prior):
    """
    Arguments: 
    freq_input - range of all possible frequencies (percepts?)
    p_back - prob of background
    p_low - prob of low condition
    log_prior - list of prior parameters
    """
    
    log_prior_low_mean = log_prior[0]; log_prior_low_sigma = log_prior[2];
    log_prior_high_mean = log_prior[1]; log_prior_high_sigma = log_prior[2];
    prior_low = gaussian(x=freq_input, mean=log_prior_low_mean, sigma=log_prior_low_sigma)
    prior_high = gaussian(x=freq_input, mean=log_prior_high_mean, sigma=log_prior_high_sigma)
    prior_dist_mixed_high = p_back*(1/len(freq_input)) + (1-p_back)*prior_high \
    #mixture model with p(T|B) = 1/no. of possible freqs
    prior_dist_mixed_high /= prior_dist_mixed_high.sum() #normalizing
    prior_dist_mixed_high = np.expand_dims(prior_dist_mixed_high, axis = 1)
    prior_dist_mixed_low = p_back*(1/len(freq_input)) + (1-p_back)*prior_low \
    #mixture model with p(T|B) = 1/no. of possible freqs
    prior_dist_mixed_low /= prior_dist_mixed_low.sum() #normalizing
    prior_dist_mixed_low = np.expand_dims(prior_dist_mixed_low, axis = 1)
        
    if n_tones == 3:
        prior_tones_low = np.expand_dims(prior_dist_mixed_low@np.transpose\
                                         (prior_dist_mixed_low),axis=2)@np.transpose(prior_dist_mixed_low) \
        #p(T1,T2..|L) 
        prior_tones_high = np.expand_dims(prior_dist_mixed_high@np.transpose\
                                          (prior_dist_mixed_high),axis=2)@np.transpose(prior_dist_mixed_high) \
        #p(T1,T2..|H) 
    elif n_tones == 1:
        prior_tones_low = prior_dist_mixed_low
        prior_tones_high = prior_dist_mixed_high
        
    normalizer = (1-p_low)*prior_tones_high + p_low*prior_tones_low #p(H)*p(T1,T2..|H) + p(L)*p(T1,T2..|L)
    posterior = prior_tones_high*(1-p_low)/normalizer
    # posterior /= np.sum(posterior)
    
    return prior_dist_mixed_high, prior_dist_mixed_low, prior_tones_high, prior_tones_low, normalizer, posterior

In [8]:
# define mle function
def MLE(params):
    log_prior_low_mean, log_prior_high_mean, log_prior_sigma, sigma_sensory, prob_back, prob_low = \
    params[0], params[1], params[2], params[3], params[4], params[5] # inputs are guesses at our parameters  
        
    _,_,LikelihoodLatentTonegivenHigh,LikelihoodLatentTonegivenLow,_,_ = \
    posterior_array(log_freq_seq_array, len(trial_tones[0]), p_back=prob_back, p_low=prob_low,\
                    log_prior=[log_prior_low_mean,log_prior_high_mean,log_prior_sigma])

    LikelihoodPerceptgivenHigh = np.zeros((len(log_freq_percept),len(log_freq_percept),len(log_freq_percept)))
    LikelihoodPerceptgivenLow = np.zeros((len(log_freq_percept),len(log_freq_percept),len(log_freq_percept)))
    
    for itrue1 in range(len(log_freq_seq_array)):
        for itrue2 in range(len(log_freq_seq_array)):
            for itrue3 in range(len(log_freq_seq_array)):
                probPerceptgivenLatentTones = Tones3dgrid([log_freq_seq_array[itrue1],\
                                                           log_freq_seq_array[itrue2],\
                                                           log_freq_seq_array[itrue3]],sigma=sigma_sensory)
                LikelihoodPerceptgivenHigh \
                += probPerceptgivenLatentTones * LikelihoodLatentTonegivenHigh[itrue1,itrue2,itrue3]
                LikelihoodPerceptgivenLow \
                += probPerceptgivenLatentTones * LikelihoodLatentTonegivenLow[itrue1,itrue2,itrue3]
    probHighgivenPercept = LikelihoodPerceptgivenHigh*(1-prob_low)/\
    (LikelihoodPerceptgivenHigh*(1-prob_low) + LikelihoodPerceptgivenLow*(prob_low))
        
    neg_ll = 0; 
    probability_high = np.zeros((len(trial_tones),1))
    for i_trial in range(len(trial_tones)):
        input_array_mat = Tones3dgrid(np.array([np.log10(trial_tones[i_trial][0]),\
                                               np.log10(trial_tones[i_trial][1]),
                                               np.log10(trial_tones[i_trial][2])]),sigma=sigma_sensory)
        probability_high0 = np.sum(np.multiply(probHighgivenPercept>0.5,input_array_mat))
        probability_high[i_trial] = np.sum(np.multiply(probHighgivenPercept>0.5,input_array_mat))
            
        if trial_behaviour[i_trial]:
            if np.isnan(np.log(probability_high0 + 0.0000001)) \
            or np.isinf(np.log(probability_high0 + 0.0000001)) \
            or np.isnan(np.log(1-probability_high0 + 0.0000001)) \
            or np.isinf(np.log(1-probability_high0 + 0.0000001)):
                pdb.set_trace()
            neg_ll += -np.log(probability_high0 + 0.0000001) # if high dist is chosen by observer
        else:
            neg_ll += -np.log(1 - probability_high0 + 0.0000001) # if low dist is chosen by observer   
    return(neg_ll, probability_high)

In [9]:
"""
New optimization algorithm: uses scipy.optimize.fmin. 
Crude grid initially and then find minimum using the function.
"""

guess_low_mean = np.arange(2.1,2.71,0.15); guess_high_mean = np.arange(2.7,3.31,0.15); 
guess_sigma = np.arange(0.05,1,0.2); guess_sensory_sigma = np.array([0.15]);
guess_p_back = np.array([0]); guess_p_low = np.arange(0.6,0.81,0.05);

# Constraining guesses of means of low and high distributions based on observed behaviour in figure shown above. 

neg_ll_array = np.zeros((len(guess_low_mean), len(guess_high_mean), len(guess_sigma), 
                         len(guess_sensory_sigma), len(guess_p_back), len(guess_p_low)))
for lm in tqdm(range(len(guess_low_mean))):
    for hm in tqdm(range(len(guess_high_mean)), leave=False, desc="High mean"):
        for s in range(len(guess_sigma)):
            for ss in range(len(guess_sensory_sigma)):
                for pb in range(len(guess_p_back)):
                    for pl in range(len(guess_p_low)):
                        params = [guess_low_mean[lm], guess_high_mean[hm], guess_sigma[s], 
                                  guess_sensory_sigma[ss], guess_p_back[pb], guess_p_low[pl]]
                        # print(lm, hm, pb)
                        neg_ll_array[lm,hm,s,ss,pb,pl],_ = MLE(params) 
                    
"""
Means and p_back corresponding to the least negative log likelihood value
"""
idxs = np.where(neg_ll_array == np.amin(neg_ll_array)) 
best_thetas = np.array([guess_low_mean[idxs[0]], guess_high_mean[idxs[1]], guess_sigma[idxs[2]], \
                        guess_sensory_sigma[idxs[3]], guess_p_back[idxs[4]], guess_p_low[idxs[5]]])

print(best_thetas)

  0%|          | 0/5 [00:00<?, ?it/s]

High mean:   0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-7-c01720b117b2>:50: RuntimeWarning: invalid value encountered in true_divide
  posterior = prior_tones_high*(1-p_low)/normalizer


High mean:   0%|          | 0/5 [00:00<?, ?it/s]

High mean:   0%|          | 0/5 [00:00<?, ?it/s]

High mean:   0%|          | 0/5 [00:00<?, ?it/s]

High mean:   0%|          | 0/5 [00:00<?, ?it/s]

[[2.1  2.1  2.1  2.1  2.1  2.1  2.25 2.25 2.25 2.25 2.25 2.25 2.25 2.25
  2.25 2.4  2.4  2.4  2.4  2.4  2.4  2.4  2.4  2.55 2.55 2.55 2.55]
 [2.7  2.85 3.15 3.3  3.3  3.3  2.7  2.85 3.   3.   3.   3.15 3.15 3.15
  3.15 2.7  2.7  2.85 2.85 3.   3.   3.   3.   2.7  2.85 2.85 2.85]
 [0.65 0.65 0.65 0.45 0.45 0.65 0.65 0.65 0.45 0.65 0.85 0.25 0.25 0.45
  0.65 0.45 0.65 0.45 0.65 0.25 0.25 0.25 0.45 0.25 0.05 0.25 0.25]
 [0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15
  0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.8  0.8  0.7  0.65 0.7  0.6  0.7  0.7  0.8  0.65 0.6  0.75 0.8  0.65
  0.6  0.7  0.6  0.7  0.6  0.7  0.75 0.8  0.6  0.65 0.8  0.6  0.65]]


In [ ]:
# define mle function
def MLE_fmin(params):
    log_prior_low_mean, log_prior_high_mean, log_prior_sigma, sigma_sensory, prob_back, prob_low = \
    params[0], params[1], params[2], params[3], params[4], params[5] # inputs are guesses at our parameters  
    
    _,_,LikelihoodLatentTonegivenHigh,LikelihoodLatentTonegivenLow,_,_ = \
    posterior_array(log_freq_seq_array, len(trial_tones[0]), p_back=prob_back, p_low=prob_low,\
                    log_prior=[log_prior_low_mean,log_prior_high_mean,log_prior_sigma])

    LikelihoodPerceptgivenHigh = np.zeros((len(log_freq_percept),len(log_freq_percept),len(log_freq_percept)))
    LikelihoodPerceptgivenLow = np.zeros((len(log_freq_percept),len(log_freq_percept),len(log_freq_percept)))
    
    for itrue1 in range(len(log_freq_seq_array)):
        for itrue2 in range(len(log_freq_seq_array)):
            for itrue3 in range(len(log_freq_seq_array)):
                probPerceptgivenLatentTones = Tones3dgrid([log_freq_seq_array[itrue1],\
                                                           log_freq_seq_array[itrue2],\
                                                           log_freq_seq_array[itrue3]],sigma=sigma_sensory)
                LikelihoodPerceptgivenHigh \
                += probPerceptgivenLatentTones * LikelihoodLatentTonegivenHigh[itrue1,itrue2,itrue3]
                LikelihoodPerceptgivenLow \
                += probPerceptgivenLatentTones * LikelihoodLatentTonegivenLow[itrue1,itrue2,itrue3]
    probHighgivenPercept = LikelihoodPerceptgivenHigh*(1-prob_low)/\
    (LikelihoodPerceptgivenHigh*(1-prob_low) + LikelihoodPerceptgivenLow*(prob_low))
        
    neg_ll = 0; 
    
    probability_high = np.zeros((len(trial_tones),1))
    for i_trial in range(len(trial_tones)):
        input_array_mat = Tones3dgrid(np.array([np.log10(trial_tones[i_trial][0]),\
                                               np.log10(trial_tones[i_trial][1]),
                                               np.log10(trial_tones[i_trial][2])]),sigma=sigma_sensory)
        probability_high0 = np.sum(np.multiply(probHighgivenPercept>0.5,input_array_mat))
        probability_high[i_trial] = np.sum(np.multiply(probHighgivenPercept>0.5,input_array_mat))
            
        if trial_behaviour[i_trial]:
            if np.isnan(np.log(probability_high0 + 0.0000001)) \
            or np.isinf(np.log(probability_high0 + 0.0000001)) \
            or np.isnan(np.log(1-probability_high0 + 0.0000001)) \
            or np.isinf(np.log(1-probability_high0 + 0.0000001)):
                pdb.set_trace()
            neg_ll += -np.log(probability_high0 + 0.0000001) # if high dist is chosen by observer
        else:
            neg_ll += -np.log(1 - probability_high0 + 0.0000001) # if low dist is chosen by observer
    print(params, neg_ll)        
    return(neg_ll)

minimum_nll = scipy.optimize.fmin(MLE_fmin, best_thetas, maxiter=10000, maxfun=10000, 
                                  xtol=0.01, ftol=0.01)

print(minimum_nll)

In [ ]:
# Influence plots the way we currently understand them (11-17-2020)
unique_tones = np.unique(trial_tones)
tone1_prob_behaviour = np.zeros((len(unique_tones)))

tone2_prob_behaviour = np.zeros((len(unique_tones)))
tone3_prob_behaviour = np.zeros((len(unique_tones)))

for i_tone in range(len(unique_tones)):
    tone1_prob_behaviour[i_tone] = np.mean(trial_behaviour[trial_tones[:,0]\
                                                       ==unique_tones[i_tone]])
    tone2_prob_behaviour[i_tone] = np.mean(trial_behaviour[trial_tones[:,1]\
                                                       ==unique_tones[i_tone]])
    tone3_prob_behaviour[i_tone] = np.mean(trial_behaviour[trial_tones[:,2]\
                                                       ==unique_tones[i_tone]])
influence1, = plt.plot(np.log10(unique_tones), tone1_prob_behaviour, label = 'Influence of Tone 1')
influence2, = plt.plot(np.log10(unique_tones), tone2_prob_behaviour, label = 'Influence of Tone 2')
influence3, = plt.plot(np.log10(unique_tones), tone3_prob_behaviour, label = 'Influence of Tone 3')
influence, = plt.plot(np.log10(unique_tones), (tone1_prob_behaviour+tone2_prob_behaviour+tone3_prob_behaviour)/3,
                       'k', label = 'Average Influence')

_, probability_high = MLE(minimum_nll)

tone1_prob_behaviour = np.zeros((len(unique_tones)))
tone2_prob_behaviour = np.zeros((len(unique_tones)))
tone3_prob_behaviour = np.zeros((len(unique_tones)))

for i_tone in range(len(unique_tones)):
    tone1_prob_behaviour[i_tone] = np.mean(probability_high[trial_tones[:,0]\
                                                       ==unique_tones[i_tone]])
    tone2_prob_behaviour[i_tone] = np.mean(probability_high[trial_tones[:,1]\
                                                       ==unique_tones[i_tone]])
    tone3_prob_behaviour[i_tone] = np.mean(probability_high[trial_tones[:,2]\
                                                       ==unique_tones[i_tone]])
mnll_influence, = plt.plot(np.log10(unique_tones),
                           (tone1_prob_behaviour+tone2_prob_behaviour+tone3_prob_behaviour)/3,'k.',
                          label = 'p(B_H|T) given fmin parameters')
"""
 _, probability_high = MLE(best_thetas)

tone1_prob_behaviour = np.zeros((len(unique_tones)))
tone2_prob_behaviour = np.zeros((len(unique_tones)))
tone3_prob_behaviour = np.zeros((len(unique_tones)))

for i_tone in range(len(unique_tones)):
    tone1_prob_behaviour[i_tone] = np.mean(probability_high[trial_tones[:,0]\
                                                       ==unique_tones[i_tone]])
    tone2_prob_behaviour[i_tone] = np.mean(probability_high[trial_tones[:,1]\
                                                       ==unique_tones[i_tone]])
    tone3_prob_behaviour[i_tone] = np.mean(probability_high[trial_tones[:,2]\
                                                       ==unique_tones[i_tone]])
grid_influence, = plt.plot(np.log10(unique_tones),
                           (tone1_prob_behaviour+tone2_prob_behaviour+tone3_prob_behaviour)/3,'k--',
                          label = 'p(B_H|T) given grid parameters')
"""
plt.legend(handles=[influence, mnll_influence])

plt.xlim([1.9,3.6])
plt.ylim([-0.2,1.1])
plt.xlabel('log10(Tones)')
plt.ylabel('p(B_H|T)')
# plt.savefig('figures/long_context/experimenter=nate_categorization_task_long_2021-01-18_16h48.56.916.png')